In [ ]:
pip install -U -q "langchain==0.0.331rc3" langchain-experimental "openai>=1.1"

In [ ]:
from dotenv import load_dotenv
from langchain_experimental.openai_assistant import OpenAIAssistantRunnable
from openai.types.beta.threads.thread_message import ThreadMessage

load_dotenv(dotenv_path="env")

interpreter_assistant_agent = OpenAIAssistantRunnable.create_assistant(
    name="langchain assistant",
    instructions="您是一位有用的私人助理。 当被问到问题时，编写并运行 Python 代码来回答问题。",
    tools=[{"type": "code_interpreter"}],
    model="gpt-4",
)
file = interpreter_assistant_agent.client.files.create(
    file=open("GDP.csv", "rb"), purpose="assistants"
)
while True:
    question = input("You: ")
    if question == "":
        break
    output = interpreter_assistant_agent.invoke(
        {"content": question, "file_ids": [file.id]}
    )
    ai = ""
    for msg in output:
        if isinstance(msg, ThreadMessage):
            for c in msg.content:
                ai = ai + "\n" + c.text.value
        else:
            print(f"Unknow Message:{msg}")
    print(f"AI:{ai}")

In [ ]:
from dotenv import load_dotenv
from openai import OpenAI

load_dotenv(dotenv_path="env")

client = OpenAI()

my_assistants = client.beta.assistants.list(
    order="desc",
    limit="20",
)
print("My assistants before cleaning:")
for assistant in my_assistants:
	print(assistant)
for assistant in my_assistants.data:
	client.beta.assistants.delete(assistant_id=assistant.id)
my_assistants = client.beta.assistants.list(
    order="desc",
    limit="20",
)
print("My assistants after cleaning:")
for assistant in my_assistants:
	print(assistant)